In [63]:
from sklearn.model_selection import cross_validate

In [72]:
# reading csv file  

import warnings
warnings.filterwarnings('ignore')
import nltk
nltk.download('stopwords')
# Modules for data manipulation
import numpy as np
import pandas as pd
import re

# Modules for visualization
import matplotlib.pyplot as plt
import seaborn as sb


from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Tools for creating ngrams and vectorizing input data
from gensim.models import Word2Vec, Phrases

# Tools for building a model
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences

# Tools for assessing the quality of model prediction
from sklearn.metrics import accuracy_score, confusion_matrix

import os
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import pandas as pd
import os
import io
import numpy as np
import zipfile
import sys
import datetime
from sklearn.model_selection import train_test_split
import requests
import shutil
import keras
from keras import optimizers
from keras import backend as K
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 

from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping

from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
import os, re, csv, math, codecs




from subprocess import check_output



def load_dataset(url: str, sep: str = ',', decode: str = 'utf-8', keep_default_na: bool = True, error_bad_lines: bool = True):
    def requests_csv(url: str, decode: str):
        return (requests.get(url).content).decode(decode)
    
    return pd.read_csv(io.StringIO(requests_csv(url, decode)),
                       sep=sep,
                       keep_default_na=keep_default_na,
                       error_bad_lines=error_bad_lines
                       )
url_dataset = "https://media.githubusercontent.com/media/Y4rd13/datasets/main/upwork_dataset.csv" 

from sklearn.model_selection import train_test_split

data =  load_dataset(url_dataset, decode="cp1252", keep_default_na=True, error_bad_lines=True)
#train_df = train_df.sample(2000)
data['jobText'] = data['jobText'].astype(str) 
data['approved'] = data['approved'].astype(int)
train, test = train_test_split(df, test_size = 0.2, random_state=42)
#Classes: ['N': 0, 'S': 1, 'V': 2, 'F': 3, 'Q': 4]
# N- normal, S-supraventricular, V-ventricular, F-fusion, Q- unknown

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [65]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

data['jobText'] = data['jobText'].map(lambda com : clean_text(com))

In [66]:
RE_PATTERNS = {
    ' fuck':
        [
            '(f)(u|[^a-z0-9 ])(c|[^a-z0-9 ])(k|[^a-z0-9 ])([^ ])*',
            '(f)([^a-z]*)(u)([^a-z]*)(c)([^a-z]*)(k)',
            ' f[!@#\$%\^\&\*]*u[!@#\$%\^&\*]*k', 'f u u c',
            '(f)(c|[^a-z ])(u|[^a-z ])(k)', r'f\*',
            'feck ', ' fux ', 'f\*\*', 
             'f\.u\.', 'f###', ' fu ', 'f@ck', 'f u c k', 'f uck', 'f ck','fuk','fk'
        ],
    ' ass ':
        [
            '[^a-z]ass ', '[^a-z]azz ', 'arrse', ' arse ', '@\$\$','[^a-z]anus', ' a\*s\*s', '[^a-z]ass[^a-z ]',
            'a[@#\$%\^&\*][@#\$%\^&\*]', '[^a-z]anal ', 'a s s'
        ],
    ' asshole ':
        [
            ' a[s|z]*wipe', 'a[s|z]*[w]*h[o|0]+[l]*e', '@\$\$hole'
        ],
    ' bitch ':
        [
            'b[w]*i[t]*ch', 'b!tch',
            'bi\+ch', 'b!\+ch', '(b)([^a-z]*)(i)([^a-z]*)(t)([^a-z]*)(c)([^a-z]*)(h)',
            'biatch', 'bi\*\*h', 'bytch', 'b i t c h'
        ],
    ' bastard ':
        [
            'ba[s|z]+t[e|a]+rd'
        ],
    ' trans gender':
        [
            'transgender'
        ],
    ' gay ':
        [
            'gay' , 'g4y'
        ],
    ' cock ':
        [
            '[^a-z]cock', 'c0ck', '[^a-z]cok ', 'c0k', '[^a-z]cok[^aeiou]', ' cawk',
            '(c)([^a-z ])(o)([^a-z ]*)(c)([^a-z ]*)(k)', 'c o c k'
        ],
    ' dick ':
        [
            ' dick[^aeiou]', 'deek', 'd i c k','dik'
        ],
    ' suck ':
        [
             '(s)([^a-z ]*)(u)([^a-z ]*)(c)([^a-z ]*)(k)', 'suck', '5uck', 's u c k'
        ],
    ' cunt ':
        [
            'cunt', 'c u n t'
        ],
    ' bullshit ':
        [
            'bullsh\*t', 'bull\$hit'
        ],
    ' idiot ':
        [
            'i[d]+io[t]+', '(i)([^a-z ]*)(d)([^a-z ]*)(i)([^a-z ]*)(o)([^a-z ]*)(t)',  'i d i o t'
                                                                                     
        ],
    ' dumb ':
        [
            '(d)([^a-z ]*)(u)([^a-z ]*)(m)([^a-z ]*)(b)'
        ],
    ' shit ':
        [
            'shitty', '(s)([^a-z ]*)(h)([^a-z ]*)(i)([^a-z ]*)(t)', 'shite', '\$hit', 's h i t'
        ],
    ' shit hole ':
        [
            'shythole'
        ],
    ' retard ':
        [
            'returd', 'retad', 'retard', 'wiktard', 'wikitud'
        ],
    ' rape ':
        [
            ' raped'
        ],
    ' dumb ass':
        [
            'dumbass', 'dubass'
        ],
    ' ass head':
        [
            'butthead'
        ],
    ' sex ':
        [
             's3x'
        ],
    ' nigger ':
        [
            'nigger', ' nigr ', 'negrito', 'niguh', 'n3gr', 'n i g g e r'
        ],
    ' nigga ':
        [
            'niga', 'ni[g]+a', ' nigg[a]+'
        ],
    ' shut the fuck up':
        [
            'stfu'
        ],
    ' pussy ':
        [
            'pussy[^c]', 'pusy', 'pussi[^l]', 'pusses'
        ],
    ' faggot ':
        [
            'faggot', ' fa[g]+[s]*[^a-z ]', 'fagot', 'f a g g o t', 'faggit',
            '(f)([^a-z ]*)(a)([^a-z ]*)([g]+)([^a-z ]*)(o)([^a-z ]*)(t)', 'fau[g]+ot', 'fae[g]+ot',
        ],
    ' motherfucker':
        [
            ' motha ', ' motha f', ' mother f', 'motherucker',
        ],
    ' whore ':
        [
             'w h o r e'
        ],
}


for target,patterns in RE_PATTERNS.items():
  for pat in patterns:
    data['jobText'] = data['jobText'].map(lambda x : re.sub(pat,target,x))

In [67]:
data['jobText'] = data['jobText'].apply(lambda x: x.lower())
# removing special chars
data['jobText'] = data['jobText'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
data['jobText'] = data['jobText'].str.replace('rt','')
data.head()

,title,jobRegion,jobText,jobExperienceLevel,jobType,clientLocation,clientRating,approved
0,Web Design & Development Team (on-going work),Worldwide,we need a team that is capable of working with...,Expert,Complex project,United States,5.0,1
1,Website development,Worldwide,we are looking for someone to build a website ...,Expert,One-time project,United States,5.0,1
2,Auction Website to sell Household items,Worldwide,i am looking to hire a agency or freelancers w...,Expert,Complex project,United States,0.0,0
3,App development (Android & iOS),Worldwide,i am looking for someone that can make a compl...,Intermediate,One-time project,Sweden,4.0,0
4,Mobile App Development (Media playback app),Worldwide,what the deliverable is an audio player mobile...,Intermediate,One-time project,United States,5.0,0


In [68]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier
import pandas as pd
import os


cv = CountVectorizer(max_features=4500, encoding="utf-8",  
      ngram_range = (1,2),  
      token_pattern = "[A-Za-z_][A-Za-z\d_]*")
X = cv.fit_transform(data.jobText).toarray()
y = data['approved']
X_train, X_test, y_train, y_test = train_test_split(X, y, 
      test_size=0.15, 
      random_state=0)
count_df = pd.DataFrame(X_train, columns=cv.get_feature_names())

count_df['etiket'] = y_train

In [69]:
data.sample(5)

,title,jobRegion,jobText,jobExperienceLevel,jobType,clientLocation,clientRating,approved
1785,Mobile App Development,Worldwide,the deliverable is a simple intuitive mobile a...,Intermediate,One-time project,United States,5.00,0
2602,App Developer,Worldwide,looking for a relatively inexpensive freelance...,Intermediate,One-time project,United States,0.00,0
2457,Networking Car App,Worldwide,want to develop a new social media and network...,Expert,Ongoing project,United States,0.00,1
3816,Looking to discuss making a tracking app for s...,Worldwide,this is a whole new concept i am looking to ha...,Expert,Ongoing project,United Kingdom,4.81,0
3762,app developer,Worldwide,i have a full figma design set for an app wher...,Intermediate,One-time project,United States,4.96,0


In [70]:
count_df.iloc[1:10,1:10]

,a app,a back,a backend,a basic,a better,a big,a bit,a bonus,a brief
1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0


In [71]:
# fit model no training data

model = XGBClassifier()
model.fit(X_train, y_train)

# make predictions for test data

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

[22:34:13] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 58.05%


# https://suatatan.com/posts/sklearn_xgboost_tc/

In [ ]:
from os import chdir
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import re
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
import xgboost as xgb
import numpy as np
import seaborn as sns

# Extracting the Dataset from Local Storage
## training set

# Pre Processing
stop_words = stopwords.words('english')
wnl = WordNetLemmatizer()

def preprocess(text_column):
    
# Remove link,user and special characters
# And Lemmatize the words
new_review = []
    for review in text_column:
        
# this text is a list of tokens for the review
        text = re.sub("@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+", ' ', str(review).lower()).strip()
        text = [wnl.lemmatize(i) for i in text.split(' ') if i not in stop_words]
        new_review.append(' '.join(text))
        return new_review

train['jobText'] = preprocess(train['jobText'])
test['jobText'] = preprocess(test['jobText'])

# Data Exploration of Training Set
## proportion of positive and negative reviews
n_positive = 0
n_negative = 0
n_ratings = Counter(train['approved'])
for i in n_ratings.keys():
    
    if i == 1:
              n_positive += n_ratings[i]
    elif i < 1:
              n_negative += n_ratings[i]
plt.bar(['Positive', 'Negative'], [n_positive, n_negative])
plt.title('Proportion of Positive and Negative Reviews')
plt.ylabel('Count')
plt.show()


## counting the frequencies of words
train_pos = train.loc[train['rating'] == 1, 'jobText']
train_neg = train.loc[train['rating'] < 1, 'jobText']

total_freq = dict()
unique_freq = dict()
for review in train_pos:
    
    
# this text is a list of tokens for the review
    text = review.split(' ')
    freq = nltk.FreqDist(text)
    for token, count in freq.most_common(20):
        
        
        if token in total_freq.keys():
            
            total_freq[token] += count
            unique_freq[token] += 1
        else:
            total_freq[token] = count
            unique_freq[token] = 1
## top 20 words for positive reviews
sorted(total_freq.items(), key = lambda x: x[1], reverse = True)[:20]
sorted(unique_freq.items(), key = lambda x: x[1], reverse = True)[:20]

total_freq = dict()
unique_freq = dict()
for review in train_neg:
    
# this text is a list of tokens for the review
    text = review.split(' ')
    freq = nltk.FreqDist(text)
    for token, count in freq.most_common(20):
        
        if token in total_freq.keys():
            
            total_freq[token] += count
            unique_freq[token] += 1
        else:
            
            total_freq[token] = count
            unique_freq[token] = 1
## top 20 words for positive reviews
sorted(total_freq.items(), key = lambda x: x[1], reverse = True)[:20]
sorted(unique_freq.items(), key = lambda x: x[1], reverse = True)[:20]

# Further Pre Processing
## splitting into X and y
## creating labels
train_labels = []
for rating in train['approved']:
    
    if rating == 1 :
        
        train_labels.append(1)
    elif rating < 1:
        train_labels.append(0)
    else:
        train_labels.append(-1)

test_labels = []
for rating in test['approved']:
    
    if rating ==1 :
        test_labels.append(1)
    elif rating < 1:
        test_labels.append(0)
    else:
        
        test_labels.append(-1)
              
## creating train_x and test_x
train_x = train['jobText']
test_x = test['jobText']
"""
What CountVectorizer does:
It creates one very large matrix with one column for every unique word in your corpus
(where the corpus is all 50k reviews in our case). Then we transform each review into 
one row containing 0s and 1s, where 1 means that the word in the corpus corresponding 
to that column appears in that review.
"""
## creates a matrix where the columns are the unique words and the each row is a review
## where the corresponding element indicate the presence of that particular word
cv = CountVectorizer(binary = True)
cv.fit(train['review'])
train_x = cv.transform(train_x)
test_x = cv.transform(test_x)

# Looking at different ways to tokenize each review
## This is for Word Count
cv = CountVectorizer(binary = False)
cv.fit(train_x)
k_train_x = cv.transform(train_x)
k_test_x = cv.transform(test_x)

## Returns the TF-IDF for each Word
tv = TfidfVectorizer(analyzer = 'word')
tv.fit(train_x)
k_train_x = tv.transform(train_x)
k_test_x = tv.transform(test_x)

## Returns the TF-IDF for each 2-gram Words
tv = TfidfVectorizer(analyzer = 'word', ngram_range = (2, 2))
tv.fit(train_x)
k_train_x = tv.transform(train_x)
k_test_x = tv.transform(test_x)

## Returns the TF-IDF for each n-gram character within Words
tv = TfidfVectorizer(analyzer = 'char_wb')
tv.fit(train_x)
k_train_x = tv.transform(train_x)
k_test_x = tv.transform(test_x)

# Logistic Regression Model Implementation
model = LogisticRegression(max_iter = train_x.shape[1])
model.fit(train_x, train_labels)
y_pred = model.predict(test_x)

## Log Reg Evaluation
accuracy_score(test_labels, y_pred) # 0.8742464801683798
f1_score(test_labels, y_pred) # 0.9150633505396528

# Decision Tree Model Implementation
tree_model = DecisionTreeClassifier() 
tree_model.fit(train_x, train_labels)
y_pred_tree = tree_model.predict(test_x)

## Decision Tree Evaluation
accuracy_score(test_labels, y_pred_tree) # 0.8943335308662157
f1_score(test_labels, y_pred_tree) # 0.927198749806417

# XGBoost
xgb_train_labels = []
for rating in train['rating']:
       if rating >= 7:
              xgb_train_labels.append(1)
       elif rating <= 4:
              xgb_train_labels.append(0)
       else:
              xgb_train_labels.append(None)

xgb_test_labels = []
for rating in test['rating']:
       if rating >= 7:
              xgb_test_labels.append(1)
       elif rating <= 4:
              xgb_test_labels.append(0)
       else:
              xgb_test_labels.append(None)
xgb_train = xgb.DMatrix(train_x, xgb_train_labels)
xgb_test = xgb.DMatrix(test_x, xgb_test_labels)
param = {'eta': 0.75,
         'max_depth': 50,
         'objective': 'binary:logitraw'}
## Training and Predicting
xgb_model = xgb.train(param, xgb_train, num_boost_round = 30)
y_pred_xgb = xgb_model.predict(xgb_test)
y_pred_xgb = np.where(np.array(y_pred_xgb) > 0.5, 1, -1)

# xgb Evaluation
accuracy_score(test_labels, y_pred_xgb) # 0.9247195373643664
f1_score(test_labels, y_pred_xgb) # 0.9483063452417704
recall_score(k_test_labels, y_pred_k_xgb)
precision_score(k_test_labels, y_pred_k_xgb)
confusion_matrix(k_test_labels, y_pred_k_xgb)

# Exploratory Methods for XGBoost
## looking at feature importance
xgb_scores = xgb_model.get_score(importance_type = 'weight')
xgb_scores = list(xgb_scores.items())
xgb_scores.sort(key = lambda x: x[1], reverse = True)
x = [i[0] for i in xgb_scores[:20]]
y = [i[1] for i in xgb_scores[:20]]
# plot
plot = sns.barplot(y, x)
plot.set_title('Top 20 Most Significant Variables', x = 0.66, weight = 'bold')
plot.set_xlabel('No. of times feature is used to split the data across all trees')

xgb_scores = xgb_model.get_score(importance_type = 'gain')
xgb_scores = list(xgb_scores.items())
xgb_scores.sort(key = lambda x: x[1], reverse = True)
x = [i[0] for i in xgb_scores[:20]]
y = [i[1] for i in xgb_scores[:20]]
# plot
plot = sns.barplot(y, x)
plot.set_title('Top 20 Most Significant Variables', x = 0.66, weight = 'bold')
plot.set_xlabel('How effective a feature is when used to split the data across all trees')

## a more streamline way would be:
xgb_model.feature_importances_